In [1]:

    var sparkConf = new SparkConf().setAppName("SVMWithSGDExample");
    var sc = new SparkContext(sparkConf);

    var path = "examples/data/mllib/sample_libsvm_data.txt";
    var data = MLUtils.loadLibSVMFile(sc, path);


 Split initial RDD into two... [60% training data, 40% testing data].


In [2]:
    var training = data.sample(false, 0.6, 11);
    training.cache();
    var test = data.subtract(training);


 Run training algorithm to build the model.


In [3]:
    var numIterations = 100;
    var model = SVMWithSGD.train(training, numIterations);


 Clear the default threshold.


In [4]:
    model.clearThreshold();


 Compute raw scores on the test set.


In [5]:
    var scoreAndLabels = test.map(function (lp, model) {
        var score = model.predict(lp.getFeatures());
        return new Tuple(score, lp.getLabel());
    }, [model]);


 Get evaluation metrics.


In [6]:
    var metrics =
        new BinaryClassificationMetrics(scoreAndLabels);
    var ret = {};
    ret.model = model;
    ret.auROC = metrics.areaUnderROC();

var result = ret;
    print("Area under ROC = " + result.auROC);


 Save and load model


In [7]:
    result.model.save(sc, "target/tmp/SVMWithSGDModel");
    var sameModel = SVMModel.load(sc, "target/tmp/SVMWithSGDModel");

    sc.stop();
